Импорты и настройка логирования

In [2]:
import logging
import zipfile
from io import BytesIO

import numpy as np
import pandas as pd
import requests
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split

# Отключаю лишние логи, оставляю только ошибки
logging.basicConfig(level=logging.ERROR, format='%(levelname)s: %(message)s')

Функция загрузки данных

In [3]:
def download_yandex_disk_file(url: str) -> bytes | None:
    """
    Загружает файл с Яндекс.Диска по публичной ссылке.
    """
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download"
    params = {'public_key': url}
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        download_url = response.json()['href']

        download_response = requests.get(download_url)
        download_response.raise_for_status()

        zip_file = zipfile.ZipFile(BytesIO(download_response.content))
        csv_filename = next(
            (name for name in zip_file.namelist() if name.endswith('.csv')),
            None
        )

        if csv_filename:
            return zip_file.read(csv_filename)
        else:
            logging.error("CSV файл не найден в архиве.")
            return None

    except Exception as e:
        logging.error(f"Ошибка при загрузке: {e}")
        return None

Функция предобработки данных

In [4]:
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Выполняет очистку данных, заполнение пропусков и кодирование.
    """
    df = df.copy()

    # Удаление лишних столбцов и символов
    if 'Unnamed: 15' in df.columns:
        df = df.drop('Unnamed: 15', axis=1)
    df.columns = df.columns.str.replace('\t', '', regex=False)

    df = df.drop_duplicates(subset=['user_id'], keep='first')

    df.dropna(
        subset=['age', 'gender', 'occupation', 'work_mode'], inplace=True
    )
    df.reset_index(drop=True, inplace=True)

    numeric_cols_with_nan = [
        'screen_time_hours', 'work_screen_hours', 'leisure_screen_hours',
        'sleep_quality_1_5', 'stress_level_0_10',
        'mental_wellness_index_0_100'
    ]
    for col in numeric_cols_with_nan:
        if col in df.columns and df[col].isnull().any():
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)

    gender_map = {'Male': 0, 'Female': 1}
    df['gender'] = df['gender'].map(gender_map)
    if df['gender'].isnull().any():
        mode_gender = df['gender'].mode()[0]
        df['gender'] = df['gender'].fillna(mode_gender)

    df = pd.get_dummies(
        df, columns=['occupation', 'work_mode'], drop_first=True
    )

    numeric_cols = df.select_dtypes(include=[np.number]).columns.drop(
        'user_id', errors='ignore'
    )
    df_num = df[numeric_cols].dropna(how="all")
    z_scores = np.abs(stats.zscore(df_num, nan_policy='omit'))
    mask = (z_scores < 3).all(axis=1)
    df_clean = df.loc[df_num.index[mask]]

    return df_clean


Основная функция с GridSearchCV и RandomForest


In [5]:
def classification_training(data: pd.DataFrame):
    """
    Выполняет весь пайплайн: предобработка, подготовка таргета,
    GridSearch и обучение Random Forest.
    """
    # Вызов функции предобработки внутри classification_training согласно ТЗ
    df = prepare_data(data)

    # 1) Трансформация таргета в бинарный вид
    df['mental_wellness_index_0_100'] = (
        df['mental_wellness_index_0_100'] >= 15
    ).astype(int)

    X = df.drop(columns=['user_id', 'mental_wellness_index_0_100'])
    y = df['mental_wellness_index_0_100']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 2) Сетка гиперпараметров
    # Не использую слишком большие значения n_estimators (ограничение ТЗ)
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4],
        'min_samples_split': [2, 5, 10]
    }

    # 3) Создание экземпляра модели
    rf = RandomForestClassifier(random_state=42)

    # 4) Поиск гиперпараметров
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=5,
        scoring='f1',
        n_jobs=-1
    )

    grid_search.fit(X_train, y_train)

    print(f"Best params: {grid_search.best_params_}")

    # 5) Финальная модель
    best_rf_model = grid_search.best_estimator_
    y_pred = best_rf_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # 6) Вывод метрик
    print(f"RF: {accuracy}; {f1}")

Запуск скрипта

In [6]:
# Ссылка на данные
yandex_disk_url = "https://disk.yandex.ru/d/NVmydyRuUzCxPw"

file_content = download_yandex_disk_file(yandex_disk_url)

if file_content:
    # Читаем сырые данные
    raw_data = pd.read_csv(
        BytesIO(file_content),
        sep='\t',
        engine='python',
        on_bad_lines='skip'
    )
    # Передаем сырые данные, внутри они будут обработаны через prepare_data
    classification_training(data=raw_data)
else:
    print("Не удалось получить данные.")

Best params: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
RF: 0.8421052631578947; 0.8461538461538461
